In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # For creating sample data

In [2]:
omi_mls_ds = xr.open_dataarray("/glade/u/home/mvoncyga/SOARS_2025/300hpa_full_monthly_omimls_tco.nc")
omi_mls_ds = omi_mls_ds.rename({'latitude': 'lat', 'longitude': 'lon'})

# shifting lon to be 0-360
omi_mls_ds['lon'] = omi_mls_ds['lon'] % 360
omi_mls_ds = omi_mls_ds.sortby('lon')

In [3]:
# Extract year and month as new coordinates
omi_mls_ds['year'] = omi_mls_ds.yearmonth.dt.year
omi_mls_ds['month'] = omi_mls_ds.yearmonth.dt.month


In [4]:
omi_mls_ds

<xarray.DataArray 'TropoO3' (yearmonth: 240, lat: 180, lon: 360)> Size: 62MB
[15552000 values with dtype=float32]
Coordinates:
  * yearmonth  (yearmonth) datetime64[ns] 2kB 2005-01-01 ... 2024-12-01
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    year       (yearmonth) int64 2kB 2005 2005 2005 2005 ... 2024 2024 2024 2024
    month      (yearmonth) int64 2kB 1 2 3 4 5 6 7 8 9 ... 4 5 6 7 8 9 10 11 12
Attributes:
    units:      Dobson Units
    name:       Tropospheric Ozone Mean, Ground to 300 hPa
    latitude:   -75.5, -59.5, -58.5, ..., 59.5, 75.5
    longitude:  -179.5, -178.5, ..., 179.5

In [8]:
# Calculate the mean for each year-month combination
monthly_yearly_mean = omi_mls_ds.groupby(['year', 'month']).mean(dim='yearmonth')
monthly_yearly_mean = monthly_yearly_mean.mean(dim={'lat','lon'})

In [9]:
# Convert to a Pandas DataFrame and pivot for the heatmap
#heatmap_data = monthly_yearly_mean.to_dataframe().unstack(level='year')['TropoO3'] # Assuming 'sample_data' is the name of your variable

# Alternatively, if your dataarray is already 2D (time, other_dim) you might be able to directly use:

#monthly_yearly_mean = monthly_yearly_mean.mean(dim={'month'})
heatmap_data = monthly_yearly_mean.unstack('month') 


ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('month',)

In [ ]:
# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='Blues', annot=True, fmt=".2f", linewidths=.5, cbar_kws={'label': 'Mean Value'}) # Customize as needed
plt.title('Year vs. Month Heatmap of Sample Data')
plt.xlabel('Year')
plt.ylabel('Month')
plt.yticks(rotation=0) # Make month labels horizontal
plt.show()
